# Task: move/migrate MySQL database created during the sessions to SQLite3

In [1]:
# Import libraries
import sqlite3
import pandas as pd
import re

## Establish connections

In [2]:
# Connect to database or create if doesn't exist
conn = sqlite3.connect('web_store.db')

In [3]:
# Create cursor
c = conn.cursor()

## Create tables from sql file

In [4]:
# Open and read structure file
file = open('web_store_structure.sql','r')
sql_file_structure = file.read()
file.close()

### Clean structure file

In [5]:
# Clean comments and unnecessary lines
structure_without_comments = re.sub(r"(SET.*)|(\/.*)|(ENGINE.*)|(ALTER.*)|(ADD.*)|(MODIFY.*)|(UNSIGNED)","",sql_file_structure)

In [6]:
# Replace id variable statements
structure_replaced_id = re.sub(r"\`id\` int\(\d+\)","id integer primary key autoincrement",structure_without_comments)

In [7]:
# Replace ints with integers
structure_replaced_ints = re.sub(r"int\(\d+\)","integer",structure_replaced_id)

In [8]:
# Replace enum with text
structure_replaced_enum = re.sub(r"enum\('([a-zA-Z_ ',]*)\)","text",structure_replaced_id)

In [9]:
# Clean all whitespace
structure_clean = re.sub(r"\n ","",structure_replaced_enum)

In [10]:
# Split file into queries 
structure = structure_replaced_enum.strip().split("NOT NULL\n)")

In [11]:
structure.pop()

''

In [12]:
# Create tables
for line in structure:
    c.execute(line + ")")

In [13]:
# Check that tables was successfully created in database
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('customers',), ('sqlite_sequence',), ('orders',), ('orders_products',), ('products',), ('suppliers',)]


## Insert data into tables from sql file

In [14]:
# Open and read data file
file_data = open('web_store_data.sql','r')
sql_file_data = file_data.read()
file_data.close()

### Clean data file

In [15]:
# Clean data file
data_clean = re.sub("(SET.*)|(\/.*)","",sql_file_data)

In [16]:
# Split file into queries 
data =data_clean.strip().split(";")
data.pop()

''

In [17]:
# Insert values in tables
for query in data:
    c.execute(query)

In [18]:
# Check data in table
c.execute("SELECT * FROM suppliers;")
print(c.fetchall())

[(1, 'Microsoft', 'Bill Gates', '+37012346789', 'bill.gates@microsoft.com'), (2, 'HP', 'Enrique Lores', '+37012346789', 'enrique.lores@hp.com'), (3, 'Apple', 'Tim Cook', '+37012346789', 'tim.cook@apple.com')]


## Commit changes

In [19]:
conn.commit()

## Close connections 

In [20]:
c.close()
conn.close()

## SQLite via Pandas

In [21]:
# Establish connection
con = sqlite3.connect('web_store.db')

In [22]:
# Perform SQL query and return pandas data frame
df_suppliers = pd.read_sql_query ('select * from suppliers', con)
df_suppliers

,id,name,contact,telephone_number,email
0,1,Microsoft,Bill Gates,+37012346789,bill.gates@microsoft.com
1,2,HP,Enrique Lores,+37012346789,enrique.lores@hp.com
2,3,Apple,Tim Cook,+37012346789,tim.cook@apple.com


In [23]:
# Perform SQL query and return pandas data frame
df_products = pd.read_sql_query ('select * from products', con)
df_products

,id,name,description,price,warranty_period,category,supplier_id
0,1,HP Elite Dragonfly G33,The HP Dragonfly Folio G3â€™s excellent design...,1900.00,12,PCs,2
1,2,Microsoft Office,Microsoft 365 is our cloud-powered productivit...,69.99,24,Software,1
2,3,Magic Mouse,"Magic Mouse is wireless and rechargeable, with...",99.00,18,Accessories,3


## Close connection

In [24]:
con.close()